# Install packages

In [8]:
!pip install gdown
!pip install torch
!pip install transformers
!pip install sentencepiece
!pip install git+https://github.com/Sultan-IH/SNLP-Group

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 3.3MB 16.8MB/s 
     |████████████████████████████████| 901kB 27.9MB/s 
     |████████████████████████████████| 1.2MB 2.9MB/s 
  Cloning https://github.com/Sultan-IH/SNLP-Group to /tmp/pip-req-build-wxrqhzsj
  Running command git clone -q https://github.com/Sultan-IH/SNLP-Group /tmp/pip-req-build-wxrqhzsj
  Created wheel for src: filename=src-0.0-cp37-none-any.whl size=4395 sha256=33b543e0a6184fb12fb28866b928b22e82c818edced41ba619e5c79c3b172190
  Stored in directory: /tmp/pip-ephem-wheel-cache-ecuoueqk/wheels/03/c8/2b/284b77eeabc3bb5535095272454f28fc6fe8722a07ce7bda94
Successfully built src


# Download dataset and model weights

In [9]:
!gdown https://drive.google.com/uc?id=1OBGCjRP2UVIj8maSDteB2oCvoqKPTuh7

Downloading...
From: https://drive.google.com/uc?id=1OBGCjRP2UVIj8maSDteB2oCvoqKPTuh7
To: /content/generator.pt
242MB [00:01, 164MB/s]


In [10]:
!rm -rf data
!rm daily_dialog.zip

!wget -O daily_dialog.zip http://yanran.li/files/ijcnlp_dailydialog.zip
!unzip daily_dialog.zip 
!rm daily_dialog.zip 

!mv ijcnlp_dailydialog data
!unzip data/train.zip -d data
!unzip data/test.zip -d data
!unzip data/validation.zip -d data

rm: cannot remove 'daily_dialog.zip': No such file or directory
--2021-06-01 12:45:27--  http://yanran.li/files/ijcnlp_dailydialog.zip
Resolving yanran.li (yanran.li)... 192.30.252.153, 192.30.252.154, 207.97.227.245
Connecting to yanran.li (yanran.li)|192.30.252.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4475921 (4.3M) [application/zip]
Saving to: ‘daily_dialog.zip’

daily_dialog.zip    100%[===================>]   4.27M  3.05MB/s    in 1.4s    

2021-06-01 12:45:29 (3.05 MB/s) - ‘daily_dialog.zip’ saved [4475921/4475921]

Archive:  daily_dialog.zip
   creating: ijcnlp_dailydialog/
  inflating: ijcnlp_dailydialog/.DS_Store  
  inflating: ijcnlp_dailydialog/dialogues_act.txt  
  inflating: ijcnlp_dailydialog/dialogues_emotion.txt  
  inflating: ijcnlp_dailydialog/dialogues_text.txt  
  inflating: ijcnlp_dailydialog/dialogues_topic.txt  
  inflating: ijcnlp_dailydialog/readme.txt  
  inflating: ijcnlp_dailydialog/test.zip  
 extracting: ijcnlp_dailydial

# Imports

In [11]:
from src.generator import Generator
from src.dataset import DailyDialogueDataset
from src.utils import print_dialogue

In [12]:
generator = Generator.from_file("generator.pt")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
dataset = DailyDialogueDataset(
      "data/test/dialogues_test.txt",
      tokenizer=generator.tokenizer,
  )

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


# Results on the test data

In [14]:
for i in range(10):
  context, real_reply = dataset[i]
  fake_reply = generator.generate(context)
  print_dialogue(context, real_reply, fake_reply=fake_reply, tokenizer=generator.tokenizer)
  print("")

PERSON A: Hey man, you wanna buy some weed? 
PERSON B:  Some what? 
PERSON A:  Weed! You know? Pot, Ganja, Mary Jane some chronic! 
PERSON B:  Oh, umm, no thanks. 
PERSON A:  I also have blow if you prefer to do a few lines. 
PERSON B:  No, I am ok, really. 
PERSON A:  Come on man! I even got dope and acid! Try some! 
PERSON B:  Do you really have all of these drugs? Where do you get them from? 
PERSON A:  I got my connections! Just tell me what you want and I ’ ll even give you one ounce for free. 
PERSON B:  Sounds good! Let ’ s see, I want. 
PERSON A:  Yeah?</s>
REAL REPLY: I want you to put your hands behind your head! You are under arrest!</s>
FAKE REPLY: I ’ m a little nervous!</s>

PERSON A: The taxi drivers are on strike again. 
PERSON B:  What for? 
PERSON A:  They want the government to reduce the price of the gasoline.</s>
REAL REPLY: It is really a hot potato.</s>
FAKE REPLY: Why?</s>

PERSON A: We've managed to reduce our energy consumption in our factory by about 15 per c

# Custom dialogue

In [20]:
generator.reset()

In [21]:
generator.reply("Hi, what's your name?", do_sample=True)

'My name is David.</s>'

In [22]:
generator.reply("Do you have a hobby?", do_sample=True)

'Yes, I like gardening.</s>'

In [23]:
generator.reply("Sounds interesting!", do_sample=True)

'What kind of gardening do you like?</s>'